In [113]:
from pyspark.sql import SparkSession

In [114]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [115]:
data = spark.read.csv('SMSSpamCollection',sep='\t',inferSchema=True,header=False)

In [116]:
data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [117]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [118]:
data.count()

5574

In [119]:
data = data.withColumnRenamed(existing='_c0',new='class').withColumnRenamed(existing='_c1',new='text')

In [120]:
data.show(truncate=False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class|text                                                                                                                                                                                                |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham  |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                                                                     |
|ham  |Ok lar... Joking wif u oni...                                                                                                                                                

In [121]:
from pyspark.sql.functions import length

In [122]:
data = data.withColumn('length',length(data['text']))

In [123]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [124]:
from pyspark.ml.feature import (Tokenizer,StopWordsRemover,
                                CountVectorizer,IDF,StringIndexer)

In [125]:
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [126]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [127]:
from pyspark.ml.feature import VectorAssembler

In [128]:
from pyspark.ml.classification import NaiveBayes

In [129]:
nb = NaiveBayes()

In [130]:
from pyspark.ml import Pipeline

In [131]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

In [132]:
cleaner = data_prep_pipe.fit(dataset=data)

In [133]:
clean_data = cleaner.transform(dataset=data)

In [134]:
clean_data.columns

Exception ignored in: <function JavaWrapper.__del__ at 0x000002403F9CCCA0>
Traceback (most recent call last):
  File "C:\Users\Abhinav94\anaconda3\lib\site-packages\pyspark\ml\wrapper.py", line 53, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'Tokenizer' object has no attribute '_java_obj'


['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_tokens',
 'c_vec',
 'tf_idf',
 'features']

In [135]:
clean_data = clean_data.select(['label','features'])

In [136]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,140...|
|  0.0|(13424,[10,53,102...|
|  0.0|(13424,[127,185,4...|
|  1.0|(13424,[1,47,121,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,117...|
|  1.0|(13424,[8,16,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,95,221...|
|  0.0|(13424,[555,1797,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,44...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [137]:
training,test = clean_data.randomSplit([0.7,0.3])

In [138]:
spam_detector = nb.fit(training)

In [139]:
test_results = spam_detector.transform(test)

In [140]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,13,...|[-606.73696885574...|[1.0,8.5427141541...|       0.0|
|  0.0|(13424,[0,1,5,20,...|[-816.97486335269...|[1.0,1.1242688641...|       0.0|
|  0.0|(13424,[0,1,11,32...|[-875.06288804778...|[1.0,8.3736699903...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-1379.0779094795...|[1.0,4.1763629263...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-1007.8408182451...|[1.0,8.8792797860...|       0.0|
|  0.0|(13424,[0,1,23,63...|[-1296.8551567382...|[1.0,8.5121981610...|       0.0|
|  0.0|(13424,[0,1,27,85...|[-1533.6249055007...|[3.00777188859863...|       1.0|
|  0.0|(13424,[0,1,30,11...|[-612.15808750201...|[1.0,5.5260083097...|       0.0|
|  0.0|(13424,[0,1,46,17...|[-1142.7747304521...|[1.22260848758587...|       1.0|
|  0.0|(13424,[0

In [141]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [142]:
acc_eval = MulticlassClassificationEvaluator()

In [143]:
acc = acc_eval.evaluate(test_results)

In [144]:
print("Acc of NB model ")
print(acc)

Acc of NB model 
0.9194030528817368


In [145]:
from pyspark.ml.classification import LogisticRegression

In [146]:
log = LogisticRegression()
spam_detector2 = log.fit(training)

In [147]:
test_results2 = spam_detector2.transform(test)

In [148]:
acc2 = acc_eval.evaluate(test_results2)

In [149]:
print("Acc of Logistic Regression ")
print(acc2)

Acc of Logistic Regression 
0.9817355467972917
